In [1]:
from src.utils import data_load
import pandas as pd
from src.s3_utils import pandas_from_csv_s3

# Data processing: Join PHQ9, GAD7 and ACE datasets together by record_id and redcap_event_name

In [2]:
data = data_load(data_keys={'phq9', 'generalized_anxiety_disorder_scale_gad7', 'ace', 'surveys', 'study_ids', 'check_in_adherence_log'})

In [3]:
outcomes = pd.merge(data['phq9'], data['generalized_anxiety_disorder_scale_gad7'],  how='outer', left_on=['record_id','redcap_event_name'], right_on = ['record_id','redcap_event_name'])

In [4]:
overall_df = pd.merge(data['ace'].drop(columns=['redcap_event_name']).dropna(), outcomes, how='left', on='record_id')

# Convert redcap_event_name to date for PHQ9, GAD7 and ACE datasets

In [5]:
# read study ids
id_df = data['study_ids'][['record_id', 'evidation_id']]
id_df.rename(columns={'evidation_id': 'user_id'}, inplace=True)

# add ids to survey
overall_df = overall_df.merge(id_df, on=['record_id'])
overall_df.user_id = overall_df.user_id.fillna(-1).astype(int)

# read check-in dates
ci_df = data['check_in_adherence_log']
cols = ['record_id'] + [col for col in ci_df.columns if col.endswith('_date')]
ci_df = ci_df[cols]

# add dates to survey
overall_df = overall_df.merge(ci_df, on=['record_id'])
overall_df['checkin_number'] = overall_df.redcap_event_name.str.split('ci_').str[1].str.split('_arm').str[0].fillna(0).astype(int)
overall_df['date'] = overall_df.apply(lambda x: x[f'checkin_{str(x.checkin_number)}_date'] if x.checkin_number > 0 else None, axis=1)
overall_df = overall_df[overall_df.columns.drop(list(overall_df.filter(regex='checkin_')))]
overall_df['date'] = pd.to_datetime(overall_df['date'])

In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
overall_df.head()

,record_id,ace_1,ace_2,ace_3,ace_4,ace_5,ace_6,ace_7,ace_8,ace_9,ace_10,ace_complete,redcap_event_name,phq9_1,phq9_2,phq9_3,phq9_4,phq9_5,phq9_6,phq9_7,phq9_8,phq9_9,phq9_10,phq9_complete,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,gad_8,generalized_anxiety_disorder_scale_gad7_complete,user_id,date
0,4,1,1,0,1.0,1,0,1,1,0,0,2,ci_1_arm_1,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2,1.0,1.0,0.0,2.0,1.0,3.0,0.0,1.0,2.0,28,2021-03-18
1,4,1,1,0,1.0,1,0,1,1,0,0,2,ci_2_arm_1,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,2,2.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,2.0,28,2021-04-01
2,4,1,1,0,1.0,1,0,1,1,0,0,2,ci_3_arm_1,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,2,1.0,1.0,0.0,2.0,2.0,3.0,1.0,1.0,2.0,28,2021-04-15
3,4,1,1,0,1.0,1,0,1,1,0,0,2,ci_4_arm_1,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,2,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,28,2021-04-29
4,4,1,1,0,1.0,1,0,1,1,0,0,2,ci_5_arm_1,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,2,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,28,2021-05-13


# Add Global survey data - PROMIS quality of life

In [7]:
promis_survey = data['surveys']
promis_survey = promis_survey.loc[promis_survey['question_text'] == 'In general, would you say your quality of life is:']
promis_survey['date'] = pd.to_datetime(promis_survey['date'])

In [8]:
new_label = dict()
# encoding: {'Excellent': 0, 'Very good': 1, 'Good': 2, 'Fair': 3}
for i, answer in enumerate(promis_survey.answer_text.unique()):
    new_label[answer] = i
promis_survey.replace({"answer_text": new_label}, inplace=True)

In [9]:
promis_survey.head()

,id,user_id,survey_id,title,event_date,from,to,created_at,updated_at,question_id,question_text,answer_text,date
214098,154994,37,11,Global survey,2021-07-13 00:00:00,2021-07-13 04:00:00,2021-07-14 04:00:00,2021-07-13 04:08:01.137807,2021-07-13 17:58:54.62095,115,"In general, would you say your quality of life...",0,2021-07-13
214099,127778,81,11,Global survey,2021-07-01 00:00:00,2021-07-01 04:00:00,2021-07-02 04:00:00,2021-07-01 04:07:32.412659,2021-07-01 11:54:47.462582,115,"In general, would you say your quality of life...",0,2021-07-01
214100,145441,110,11,Global survey,2021-07-09 00:00:00,2021-07-09 04:00:00,2021-07-10 04:00:00,2021-07-09 04:08:00.721924,2021-07-09 12:01:56.842804,115,"In general, would you say your quality of life...",0,2021-07-09
214101,132847,94,11,Global survey,2021-07-03 00:00:00,2021-07-03 05:00:00,2021-07-04 05:00:00,2021-07-03 05:07:42.444653,2021-07-03 20:32:52.358196,115,"In general, would you say your quality of life...",0,2021-07-03
214102,127773,55,11,Global survey,2021-07-01 00:00:00,2021-07-01 04:00:00,2021-07-02 04:00:00,2021-07-01 04:07:32.076611,2021-07-01 15:03:31.756573,115,"In general, would you say your quality of life...",0,2021-07-01


In [10]:
overall_df = overall_df.merge(promis_survey, how='outer', on=['user_id', 'date'])

In [11]:
overall_df.head()

,record_id,ace_1,ace_2,ace_3,ace_4,ace_5,ace_6,ace_7,ace_8,ace_9,ace_10,ace_complete,redcap_event_name,phq9_1,phq9_2,phq9_3,phq9_4,phq9_5,phq9_6,phq9_7,phq9_8,phq9_9,phq9_10,phq9_complete,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,gad_8,generalized_anxiety_disorder_scale_gad7_complete,user_id,date,id,survey_id,title,event_date,from,to,created_at,updated_at,question_id,question_text,answer_text
0,4.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,ci_1_arm_1,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,2.0,1.0,3.0,0.0,1.0,2.0,28,2021-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,ci_2_arm_1,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,2.0,28,2021-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,postnatal_ci_2_arm_1,1.0,0.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0,1.0,1.0,1.0,3.0,0.0,2.0,2.0,28,2021-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,ci_3_arm_1,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,2.0,2.0,3.0,1.0,1.0,2.0,28,2021-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,postnatal_ci_3_arm_1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,28,2021-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Process PHQ9, GAD and PROMIS data by taking the average over time for each individual

In [12]:
phq9_mean_lst = [f'phq9_{x}_mean' for x in range(1,11)]
gad_mean_lst = [f'gad_{x}_mean' for x in range(1,9)]
processed_overall_df = pd.DataFrame(columns=['user_id', 'ace_sum', 'promis_mean'] + phq9_mean_lst + gad_mean_lst)
for rid in overall_df['user_id'].unique():
    each_df = overall_df.loc[overall_df['user_id']==rid]
    ace_lst = [f'ace_{x}' for x in range(1,11)]
    phq9_lst = [f'phq9_{x}' for x in range(1,11)]
    gad_lst = [f'gad_{x}' for x in range(1,9)]
    ace_sum = each_df[ace_lst].mean(axis=0).sum()
    promis_mean = each_df['answer_text'].mean()
    phq9_mean = each_df[phq9_lst].mean(axis=0).rename(lambda x: x + '_mean', axis='columns').to_dict()
    gad_mean = each_df[gad_lst].mean(axis=0).rename(lambda x: x + '_mean', axis='columns').to_dict()
    processed_overall_df = processed_overall_df.append({'user_id': rid, 'ace_sum': ace_sum, 'promis_mean':promis_mean, **phq9_mean, **gad_mean}, ignore_index=True)


In [28]:
processed_overall_df.head()

,user_id,ace_sum,promis_mean,phq9_1_mean,phq9_2_mean,phq9_3_mean,phq9_4_mean,phq9_5_mean,phq9_6_mean,phq9_7_mean,phq9_8_mean,phq9_9_mean,phq9_10_mean,gad_1_mean,gad_2_mean,gad_3_mean,gad_4_mean,gad_5_mean,gad_6_mean,gad_7_mean,gad_8_mean
0,28.0,6.0,1.222222,0.071429,0.000000,2.357143,2.785714,0.214286,0.142857,0.500000,0.5,0.000000,1.000000,1.428571,0.785714,0.428571,1.214286,1.000000,2.571429,0.142857,1.000000
1,29.0,4.0,1.428571,0.923077,0.692308,0.923077,1.538462,1.307692,0.538462,0.769231,0.5,0.076923,1.384615,1.615385,1.076923,1.153846,1.615385,1.461538,1.538462,0.461538,1.384615
2,30.0,0.0,1.333333,0.083333,0.166667,0.500000,1.583333,0.000000,0.083333,0.083333,0.0,0.000000,0.454545,0.916667,0.250000,0.500000,0.083333,0.083333,0.916667,0.500000,0.454545
3,31.0,4.0,1.500000,0.333333,0.666667,2.666667,1.666667,1.000000,1.000000,0.666667,0.0,0.000000,0.333333,2.000000,2.333333,2.333333,2.333333,1.000000,2.333333,1.333333,0.333333
4,1018.0,0.0,NaN,0.000000,0.000000,0.153846,0.384615,0.000000,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000
